# 主題 05-1. 用不同方式使用 Sequential 及學習建立第一個轉移學習模型

讓我們回顧一下生命中第一個做出來的神經網路...

## 1. 初始準備

In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Keras functions
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

# Keras dataset
from keras.datasets import mnist

# Keras utilis function
from keras.utils import np_utils

C:\Users\oroch\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 2. 讀入 MNIST 數據庫

### 2.1 由 Keras 讀入 MNIST
Keras 很貼心的幫我們準備好 MNIST 數據庫, 我們可以這樣讀進來 (第一周課程中已經讀過)。

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 10s 1us/step


資料的長相

In [5]:
print("總共有 %d 訓練資料，每筆資料的尺寸為 %d x %d" %x_train.shape)
print("總共有 %d 測試資料，每筆資料的尺寸為 %d x %d" %x_test.shape)

總共有 60000 訓練資料，每筆資料的尺寸為 28 x 28
總共有 10000 測試資料，每筆資料的尺寸為 28 x 28


### 2.3 輸入格式整理

我們現在要用標準神經網路學學手寫辨識。原來的每筆數據是個 28x28 的矩陣 (array), 但標準神經網路只吃「平平的」, 也就是每次要 28x28=784 長的向量。因此我們要用 `reshape` 調校一下。

In [6]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

為了後面需要，我們把訓練/測試資料中的數字 0, 1 資料，複製一份出來

In [7]:
x_train_01 = x_train[y_train <= 1]
x_test_01 = x_test[y_test <= 1]

並將 label 轉換成 one-hot encoding 的形式

In [8]:
y_train_10 = np_utils.to_categorical(y_train, 10)
y_test_10 = np_utils.to_categorical(y_test, 10)

y_train_01 = y_train[y_train <= 1]
y_train_01 = np_utils.to_categorical(y_train_01, 2)

y_test_01 = y_test[y_test <= 1]
y_test_01 = np_utils.to_categorical(y_test_01, 2)

養成良好的習慣，適時的確認資料的大小以確保資料的一致性

In [9]:
x_train_01.shape, x_test_01.shape

((12665, 784), (2115, 784))

In [10]:
y_train_01.shape, y_test_01.shape

((12665, 2), (2115, 2))

## 3. 回顧一下 Sequential API

在一開始的時候，我們以下列的方式建立了一個具有下列設定

* 使用 <span style="color:red;">2</span> 個 hidden layers
* 每個 hidden layer 用 <span style="color:red;">500</span> 個神經元
* Activation Function 唯一指名 <span style="color:red;">sigmoid</span>

的神經網路，建立指令是透過建立 `Sequential()` 和 `.add` 的方式逐層建立，如下：

In [11]:
# Construct a sandbox to put layers inside
model = Sequential()

# Put fully-connected layers (Dense) inside 
model.add(Dense(500, input_dim=784))
model.add(Activation('sigmoid'))
model.add(Dense(500))
model.add(Activation('sigmoid'))
model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               392500    
_________________________________________________________________
activation_1 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_2 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5010      
_________________________________________________________________
activation_3 (Activation)    (None, 10)                0         
Total params: 648,010
Trainable params: 648,010
Non-trainable params: 0
_________________________________________________________________


### 3.1 觀察 model.layers

觀察 `model.layers`，可以發現 `model` 其實就是一堆神經網路層疊起來。

In [12]:
model.layers

換言之，剛剛每一個 `.add` 其實在做的事情就是：

`model.add(Dense(500, input_dim=784))` 是將 `<keras.layers.core.Activation at 0xe558ef0>` 加進 model.layers

`model.add(Activation('sigmoid'))` 是將 `<keras.layers.core.Dense at 0xe58a278>` 加入 model.layers

`model.add(Dense(500))` 是將 `<keras.layers.core.Dense at 0xe58a278>` 加入 model.layers

`model.add(Activation('sigmoid'))` 是將 `<keras.layers.core.Activation at 0xe558d68>` 加入 model.layers

`model.add(Dense(10))` 是將 `<keras.layers.core.Activation at 0xe558d68>` 加入 model.layers

`model.add(Activation('softmax'))` 是將 `<keras.layers.core.Activation at 0xe58a898>` 加入 model.layers

* 這邊的 at 0xe558ef0 代表的是記憶體位置，每次執行都會不一樣，所以和上面結果不同是正常的。

### 3.2 以 list 的形式使用 Sequential API

換言之，神經網路其實就是將隱藏層逐層堆疊在一起的 list，因此，我們也可以 list 的形式來建立相同的神經網路。

首先，我們將兩個隱藏層及其 Activation Function 分別寫在 list 中，如下：

In [13]:
first_layer = [Dense(500, input_dim=784), 
               Activation('sigmoid')]

second_layer = [Dense(500), 
                Activation('sigmoid')]

output_layer = [Dense(10), 
                Activation('softmax')]

從基本的 Python 資料結構中，我們知道 list 可以用 `+` 來進行合併，所以我們先來看看這三個 list 合併後的樣子。 

In [14]:
first_layer + second_layer + output_layer

合併起來的 list 看起來就像是**某個** `model.layers` 一樣，我們接著要做的，就讓這個 list **真的**變成某個神經網路的 `.layers`

很簡單，只要將寫成 list 的隱藏層 `+` 起來送進 `Sequential` 中即可。

In [15]:
model = Sequential(first_layer + second_layer + output_layer)

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 500)               392500    
_________________________________________________________________
activation_4 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_5 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5010      
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
Total params: 648,010
Trainable params: 648,010
Non-trainable params: 0
_________________________________________________________________


Q: 用 `.add` 和用 list 寫法建立的神經網路之差異？

A: 沒有任何差別，前者可以很直覺的建立神經網路，但後者則為使用轉移學習(transfer Learning)的手法之一，前者雖也可做，但較麻煩。

## 4. 情境題:假設我們手上有一個好棒棒的 MNIST 手寫辨識模型，但我今天想建立可以辨識 0 或 1 的模型，除了最後一層，想沿用前兩層的網路設定及結構，我該怎麼做？

首先，我們準備一個上面一樣的神經網路手寫辨識模型，除了最後一層之外都被包在一起。

In [17]:
all_except_last = [Dense(500, input_dim=784), 
                   Activation('sigmoid'),
                   Dense(500), 
                   Activation('sigmoid')]

output_layer = [Dense(10), 
                Activation('softmax')]

model_0_to_9 = Sequential(all_except_last + output_layer)
model_0_to_9.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 500)               392500    
_________________________________________________________________
activation_7 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_8 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5010      
_________________________________________________________________
activation_9 (Activation)    (None, 10)                0         
Total params: 648,010
Trainable params: 648,010
Non-trainable params: 0
_________________________________________________________________


建立完成後，我們讀取第一周已經訓練好的神經網路權重。

若同學們沒有 `handwriting_model_weights.h5` 這份模型的權重檔，請至 https://github.com/yenlung/Deep-Learning-MOOC 下載

In [18]:
model_0_to_9.load_weights('handwriting_model_weights.h5')

由於我們沒有要真的使用這個手寫辨識模型，所以不需要 compile、fit、predict 或 evaluate。

接著，我們定義新的 output layer。

In [19]:
new_output_layer = [Dense(2), 
                    Activation('softmax')]

model_0_to_1 = Sequential(all_except_last + new_output_layer)
model_0_to_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 500)               392500    
_________________________________________________________________
activation_7 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_8 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 1002      
_________________________________________________________________
activation_10 (Activation)   (None, 2)                 0         
Total params: 644,002
Trainable params: 644,002
Non-trainable params: 0
_________________________________________________________________


要注意的是，如果我們僅沿用而不訓練到前兩層，我們可以透過下面的方式將借過來的神經網路 **冷凍** 起來

In [20]:
for layer in all_except_last:
    layer.trainable = False

**冷凍**後的神經網路的 summary 會有些變化，你有發現嗎? ：)

In [21]:
model_0_to_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 500)               392500    
_________________________________________________________________
activation_7 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_8 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 1002      
_________________________________________________________________
activation_10 (Activation)   (None, 2)                 0         
Total params: 644,002
Trainable params: 1,002
Non-trainable params: 643,000
_________________________________________________________________


接著，我們來訓練這個(有一部分架構及權重跟別人借用的) 0 或 1 手寫辨識模型

In [22]:
model_0_to_1.compile(loss='mse', optimizer=SGD(lr=0.1), metrics=['accuracy'])

## 5. 訓練你的第一個透過轉移學習學到的神經網路

恭喜! 我們完成了第一個 transfer leraning 的神經網路。這裡我們還有兩件事要決定:

* 一次要訓練幾筆資料 (`batch_size`), 我們就 100 筆調一次參數好了
* 這 ~~6 萬筆資料~~ 12665 筆資料一共要訓練幾次 (`epochs`), 我們訓練個 5 次試試 (因為只剩 0 或 1的資料了，訓練太多容易 over-fitting)

於是最精彩的就來了。你要有比第一周快上100倍的心理準備... (這是因為訓練資料只剩下 1/5，且**可訓練**權重數量從 64萬變成 1千)

In [23]:
x_train_01.shape, y_train_01.shape

((12665, 784), (12665, 2))

In [24]:
model_0_to_1.fit(x_train_01, y_train_01, batch_size=100, epochs=5)

Epoch 1/5
12665/12665 [==============================] - 1s 79us/step - loss: 0.0183 - acc: 0.9806
Epoch 2/5
12665/12665 [==============================] - 0s 29us/step - loss: 0.0032 - acc: 0.9976
Epoch 3/5
12665/12665 [==============================] - 0s 29us/step - loss: 0.0026 - acc: 0.9978
Epoch 4/5
12665/12665 [==============================] - 0s 26us/step - loss: 0.0023 - acc: 0.9979
Epoch 5/5
12665/12665 [==============================] - 0s 26us/step - loss: 0.0021 - acc: 0.9979


In [25]:
score = model_0_to_1.evaluate(x_test_01, y_test_01)

2115/2115 [==============================] - 0s 46us/step


In [26]:
print('測試資料的 loss:', score[0])
print('測試資料正確率:', score[1])

測試資料的 loss: 0.0014183477335526931
測試資料正確率: 0.9985815602836879


## 6. 恭喜你完成了第一個透過轉移學習得到的神經網路模型！

雖然這個模型看起來很隨便，但轉移學習的模型**差不多**都是這樣建立的，實際上， Keras 亦提供許多被證實有良好表現且訓練好 (pre-trained) 的模型，如:

* Xception
* VGG16
* VGG19
* ResNet50
* InceptionV3
* InceptionResNetV2
* MobileNet
* DenseNet
* NASNet

詳細的使用方式可參考 Keras Documentation: https://keras.io/applications/

但使用這些模型進行轉移學習，**可能**需要其他更彈性的神經網路寫法，更多神經網路的建構技巧，待下次課程繼續。